In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
from numpy.linalg import norm

class EventMatrix:
    def __init__(self,eventexcel):
        self.eventexcel = eventexcel

    def CosSimcal(self,event1,event2):
        cosine_similarity = np.dot(event1, event2)/(norm(event1)*norm(event2))
        return cosine_similarity

    def PersonSimcal(self,event1,event2):
        correlation_matrix = np.corrcoef(event1,event2)
        pearson_correlation = correlation_matrix[0, 1]
        return pearson_correlation

    def Netting(self):
        df_target = pd.DataFrame(columns=['startevent','endevent','weight'])
        num_columns = self.eventexcel.shape[1]
        df = self.eventexcel
        for i in range(1,num_columns):
            for j in range(1,num_columns):
                if i != j:
                    weight = abs(self.CosSimcal(df.iloc[:,i],df.iloc[:,j]))
                    new_data = {'startevent': df.columns[i], 'endevent': df.columns[j], 'weight': weight}
                    df_target = pd.concat([df_target, pd.DataFrame([new_data])], ignore_index=True)
                    # print(new_data)
        return df_target

    def NettingMatrix(self):
        targetpair = self.Netting()
        G = nx.Graph()
        for startname, endname, wgt in zip(targetpair['startevent'], targetpair['endevent'], targetpair['weight']):
            G.add_edge(startname, endname, weight=wgt)
        adj_matrix = nx.to_numpy_matrix(G)
        return adj_matrix

if __name__ == '__main__':
    data = pd.read_excel('event_1.xlsx')
    event = EventMatrix(data)
    print(event.NettingMatrix())